# Try using `climpred` to recreate some results from the CanESM5 decadal prediction paper

In [1]:
import cftime

import numpy as np
import xarray as xr

from src import plot
import matplotlib.pyplot as plt

import climpred
from climpred import HindcastEnsemble

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [3]:
xr.set_options(keep_attrs=True)

## Load some data to process

In [12]:
DATA_DIR = "../../data/processed/"

hcst_dataset = "CanESM5.annual.anom_1991-2020"
hist_dataset = "CanESM5_hist.annual.anom_1991-2020"
obsv_dataset = "HadISST.annual.anom_1991-2020"

variable = "sst"

In [13]:
hcst = xr.open_zarr(f"{DATA_DIR}/{hcst_dataset}.{variable}.zarr")
hist = xr.open_zarr(
    f"{DATA_DIR}/{hist_dataset}.{variable}.zarr"
).unify_chunks()  # can delete
obsv = xr.open_zarr(f"{DATA_DIR}/{obsv_dataset}.{variable}.zarr")

hcst = hcst.drop("time")  # can delete
hcst.lead.attrs = {"units": "months"}  # can delete

/jobfs/35942650.gadi-pbs/ipykernel_151649/4147678962.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  hcst = xr.open_zarr(f"{DATA_DIR}/{hcst_dataset}.{variable}.zarr")


GroupNotFoundError: group not found at path ''

In [6]:
hcst_full = xr.open_zarr(f"{DATA_DIR}/{hcst_dataset}.{variable}.annual.full.zarr")
hist_full = xr.open_zarr(
    f"{DATA_DIR}/{hist_dataset}.{variable}.annual.full.zarr"
).unify_chunks()  # can delete
obsv_full = xr.open_zarr(f"{DATA_DIR}/{obsv_dataset}.{variable}.annual.full.zarr")

hcst_full = hcst_full.drop("time")
hcst_full.lead.attrs = {"units": "months"}  # can delete

In [7]:
def global_mean(ds):
    return ds.weighted(ds["area"]).mean(["lon", "lat"]).compute()

In [8]:
hcst_gbl = global_mean(hcst)
hist_gbl = global_mean(hist)
obsv_gbl = global_mean(obsv)

In [9]:
hcst_full_gbl = global_mean(hcst_full)
hist_full_gbl = global_mean(hist_full)
obsv_full_gbl = global_mean(obsv_full)

ValueError: `weights` cannot contain missing values. Missing values can be replaced by `weights.fillna(0)`.

## How does climpred bias correction compare to anomalisation?

In [ ]:
hindcast = HindcastEnsemble(hcst)
hindcast = hindcast.add_observations(obsv)
hindcast = hindcast.add_uninitialized(hist.chunk({"time": -1}))  # can delete

hindcast_gbl = HindcastEnsemble(hcst_gbl)
hindcast_gbl = hindcast_gbl.add_observations(obsv_gbl)
hindcast_gbl = hindcast_gbl.add_uninitialized(hist_gbl)

In [ ]:
hindcast_full_gbl = HindcastEnsemble(hcst_full_gbl)
hindcast_full_gbl = hindcast_full_gbl.add_observations(obsv_full_gbl)
hindcast_full_gbl = hindcast_full_gbl.add_uninitialized(hist_full_gbl)

hindcast_cpbc_gbl = hindcast_full_gbl.remove_bias(
    alignment="same_verifs", how="additive_mean"
)

In [ ]:
fig = plot.hindcasts(
    {"CAFE-f6 bias corrected": hindcast_cpbc_gbl.get_initialized().mean("member")},
    {"Observations": hindcast_cpbc_gbl.get_observations()},
    shade=True,
)
fig.suptitle("climpred bias correction")

fig = plot.hindcasts(
    {"CAFE-f6 anomalies": hindcast_gbl.get_initialized().mean("member")},
    {"Observed anomalies": hindcast_gbl.get_observations()},
    {"CAFE historical anomalies": hindcast_gbl.get_uninitialized().mean("member")},
    shade=True,
)
_ = fig.suptitle("1991-2020 anomalies")

## Compute a skill score

In [10]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [11]:
acc = hindcast.verify(
    reference="uninitialized",
    metric="pearson_r",
    comparison="e2o",
    alignment="same_verifs",
    dim=["init"],
)

NameError: name 'hindcast' is not defined

In [68]:
test

<xarray.Dataset>
Dimensions:  (skill: 2, lead: 10, lat: 90, lon: 144)
Coordinates:
  * lat      (lat) float64 -89.49 -87.98 -85.96 -83.93 ... 85.96 87.98 89.49
  * lead     (lead) int64 11 23 35 47 59 71 83 95 107 119
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
    zsurf    (lat, lon) float32 2.83e+03 2.83e+03 2.83e+03 ... 0.0 0.0 0.0
    area     (lat, lon) float32 2.758e+08 2.758e+08 ... 2.758e+08 2.758e+08
    month    int64 ...
  * skill    (skill) <U13 'initialized' 'uninitialized'
Data variables:
    sst      (skill, lead, lat, lon) float64 dask.array<chunksize=(1, 1, 90, 90), meta=np.ndarray>
Attributes:
    prediction_skill_software:     climpred https://climpred.readthedocs.io/
    skill_calculated_by_function:  HindcastEnsemble.verify()
    number_of_initializations:     37
    number_of_members:             40
    alignment:                     same_verifs
    metric:                        pearson_r
    comparison:                    e2o
    dim:                           ['init']
    reference:                     ['uninitialized']